In [1]:
# ══════════════════════════════════════════════════════════════════════════════
# COMPLETE EMOTION RECOGNITION TRAINING
# Using Your emotions-dataset
# ══════════════════════════════════════════════════════════════════════════════

import os
import torch
import numpy as np
import librosa
from pathlib import Path
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
from transformers import (
    Wav2Vec2ForSequenceClassification,
    Wav2Vec2FeatureExtractor,
    TrainingArguments,
    Trainer
)
from sklearn.metrics import accuracy_score, f1_score
import warnings
warnings.filterwarnings('ignore')

# ══════════════════════════════════════════════════════════════════════════════
# CONFIGURATION
# ══════════════════════════════════════════════════════════════════════════════

MODEL_NAME = "facebook/wav2vec2-xls-r-300m"
OUTPUT_DIR = "./emotion-recognition-model"
NUM_EPOCHS = 15
BATCH_SIZE = 4
LEARNING_RATE = 2e-5
MAX_LENGTH = 5 * 16000  # 5 seconds at 16kHz

# Emotion labels based on your dataset structure
EMOTION_LABELS = {
    "angry": 0,
    "happy": 1,
    "hesitant": 2,
    "interested": 3,
    "neutral": 4
}

LABEL_TO_ID = EMOTION_LABELS
ID_TO_LABEL = {v: k for k, v in EMOTION_LABELS.items()}

print(f"🎯 Emotion Labels: {EMOTION_LABELS}")
print(f"📊 Number of classes: {len(EMOTION_LABELS)}")

# ══════════════════════════════════════════════════════════════════════════════
# DATA LOADING
# ══════════════════════════════════════════════════════════════════════════════

def load_audio_file(file_path, target_sr=16000):
    """Load and preprocess audio file"""
    try:
        audio, sr = librosa.load(file_path, sr=target_sr)
        return audio
    except Exception as e:
        print(f"Error loading {file_path}: {e}")
        return None

def load_dataset_from_folders(base_path):
    """
    Load dataset from your Kaggle folder structure:
    /kaggle/input/emotions-dataset/Emotions/Angry/*.wav
    /kaggle/input/emotions-dataset/Emotions/Happy/*.wav
    etc.
    """
    data = []
    emotions_path = base_path / "Emotions"
    
    print(f"\n📂 Loading data from: {emotions_path}")
    
    # Map folder names to standardized emotion labels
    folder_to_emotion = {
        "Angry": "angry",
        "Happy": "happy",
        "Hesitant": "hesitant",
        "Interested": "interested",
        "Neutral": "neutral"
    }
    
    for folder_name, emotion in folder_to_emotion.items():
        emotion_folder = emotions_path / folder_name
        
        if not emotion_folder.exists():
            print(f"⚠️  Warning: {emotion_folder} not found!")
            continue
        
        # Find all audio files (wav, mp3, etc.)
        audio_files = list(emotion_folder.glob("*.wav")) + \
                     list(emotion_folder.glob("*.mp3")) + \
                     list(emotion_folder.glob("*.m4a"))
        
        print(f"  {emotion}: {len(audio_files)} files")
        
        for audio_file in audio_files:
            data.append({
                "path": str(audio_file),
                "emotion": emotion,
                "label": EMOTION_LABELS[emotion]
            })
    
    print(f"\n✅ Total samples loaded: {len(data)}")
    return data

# Load your dataset
DATASET_PATH = Path("/kaggle/input/emotions-dataset")
all_data = load_dataset_from_folders(DATASET_PATH)

# Split into train/val/test
train_data, temp_data = train_test_split(all_data, test_size=0.3, random_state=42, stratify=[d["label"] for d in all_data])
val_data, test_data = train_test_split(temp_data, test_size=0.5, random_state=42, stratify=[d["label"] for d in temp_data])

print(f"\n📊 Dataset splits:")
print(f"  Train: {len(train_data)} samples")
print(f"  Validation: {len(val_data)} samples")
print(f"  Test: {len(test_data)} samples")

# ══════════════════════════════════════════════════════════════════════════════
# FEATURE EXTRACTION
# ══════════════════════════════════════════════════════════════════════════════

feature_extractor = Wav2Vec2FeatureExtractor.from_pretrained(MODEL_NAME)

def preprocess_function(examples):
    """Process audio files for the model"""
    audio_arrays = []
    
    for path in examples["path"]:
        audio = load_audio_file(path)
        if audio is not None:
            # Pad or truncate to MAX_LENGTH
            if len(audio) > MAX_LENGTH:
                audio = audio[:MAX_LENGTH]
            else:
                audio = np.pad(audio, (0, MAX_LENGTH - len(audio)))
            audio_arrays.append(audio)
        else:
            # If file fails to load, use silence
            audio_arrays.append(np.zeros(MAX_LENGTH))
    
    # Extract features
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=16000,
        padding=True,
        return_tensors="pt"
    )
    
    return {
        "input_values": inputs.input_values,
        "labels": examples["label"]
    }

# Create HuggingFace datasets
train_dataset = Dataset.from_list(train_data)
val_dataset = Dataset.from_list(val_data)
test_dataset = Dataset.from_list(test_data)

dataset_dict = DatasetDict({
    "train": train_dataset,
    "validation": val_dataset,
    "test": test_dataset
})

# Apply preprocessing
print("\n🔄 Preprocessing datasets...")
dataset_dict = dataset_dict.map(
    preprocess_function,
    batched=True,
    batch_size=8,
    remove_columns=["path", "emotion"]
)

print("✅ Preprocessing complete!")

# ══════════════════════════════════════════════════════════════════════════════
# MODEL SETUP
# ══════════════════════════════════════════════════════════════════════════════

model = Wav2Vec2ForSequenceClassification.from_pretrained(
    MODEL_NAME,
    num_labels=len(EMOTION_LABELS),
    label2id=LABEL_TO_ID,
    id2label=ID_TO_LABEL,
    ignore_mismatched_sizes=True
)

# Freeze feature extractor
model.freeze_feature_encoder()

print(f"\n🤖 Model loaded: {MODEL_NAME}")
print(f"📊 Number of parameters: {model.num_parameters():,}")

# ══════════════════════════════════════════════════════════════════════════════
# TRAINING CONFIGURATION
# ══════════════════════════════════════════════════════════════════════════════

def compute_metrics(eval_pred):
    """Compute accuracy and F1 score"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    
    acc = accuracy_score(labels, predictions)
    f1 = f1_score(labels, predictions, average='weighted')
    
    return {
        "accuracy": acc,
        "f1": f1
    }

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    eval_strategy="epoch",
    save_strategy="epoch",
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    num_train_epochs=NUM_EPOCHS,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
    fp16=True,  # Use mixed precision for faster training
    save_total_limit=2,
    report_to=["tensorboard"]
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_dict["train"],
    eval_dataset=dataset_dict["validation"],
    compute_metrics=compute_metrics,
)

print("\n🚀 Starting training...")
print(f"⏱️  Epochs: {NUM_EPOCHS}")
print(f"📦 Batch size: {BATCH_SIZE}")
print(f"📈 Learning rate: {LEARNING_RATE}")

# ══════════════════════════════════════════════════════════════════════════════
# TRAIN THE MODEL
# ══════════════════════════════════════════════════════════════════════════════

trainer.train()

print("\n✅ Training complete!")

# ══════════════════════════════════════════════════════════════════════════════
# EVALUATION
# ══════════════════════════════════════════════════════════════════════════════

print("\n📊 Evaluating on test set...")
test_results = trainer.evaluate(dataset_dict["test"])

print("\n" + "="*60)
print("FINAL TEST RESULTS")
print("="*60)
for key, value in test_results.items():
    print(f"{key}: {value:.4f}")
print("="*60)

# ══════════════════════════════════════════════════════════════════════════════
# SAVE MODEL
# ══════════════════════════════════════════════════════════════════════════════

final_model_path = f"{OUTPUT_DIR}/final"
trainer.save_model(final_model_path)
feature_extractor.save_pretrained(final_model_path)

print(f"\n💾 Model saved to: {final_model_path}")
print("\n🎉 All done! Your emotion recognition model is ready!")


2026-01-29 17:15:36.709056: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1769706936.903692      55 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1769706936.958986      55 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1769706937.422881      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769706937.422929      55 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1769706937.422932      55 computation_placer.cc:177] computation placer alr

🎯 Emotion Labels: {'angry': 0, 'happy': 1, 'hesitant': 2, 'interested': 3, 'neutral': 4}
📊 Number of classes: 5

📂 Loading data from: /kaggle/input/emotions-dataset/Emotions
  angry: 499 files
  happy: 500 files
  hesitant: 502 files
  interested: 498 files
  neutral: 500 files

✅ Total samples loaded: 2499

📊 Dataset splits:
  Train: 1749 samples
  Validation: 375 samples
  Test: 375 samples


preprocessor_config.json:   0%|          | 0.00/212 [00:00<?, ?B/s]


🔄 Preprocessing datasets...


Map:   0%|          | 0/1749 [00:00<?, ? examples/s]

...

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-xls-r-300m and are newly initialized: ['classifier.bias', 'classifier.weight', 'projector.bias', 'projector.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



🤖 Model loaded: facebook/wav2vec2-xls-r-300m
📊 Number of parameters: 315,702,405

🚀 Starting training...
⏱️  Epochs: 15
📦 Batch size: 4
📈 Learning rate: 2e-05


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,1.602900,1.599049,0.197333,0.070883
2,1.356900,1.230649,0.597333,0.548685
3,0.908100,0.799255,0.749333,0.710771
4,0.798300,0.646883,0.792000,0.788147
5,0.577300,0.502910,0.810667,0.802229
6,0.533400,0.517490,0.805333,0.804103
7,0.514800,0.561902,0.786667,0.786075
8,0.387700,0.567250,0.762667,0.761251
9,0.215400,0.358186,0.880000,0.880687
10,0.220000,0.391238,0.872000,0.870476



✅ Training complete!

📊 Evaluating on test set...



FINAL TEST RESULTS
eval_loss: 0.4150
eval_accuracy: 0.8613
eval_f1: 0.8641
eval_runtime: 36.5548
eval_samples_per_second: 10.2590
eval_steps_per_second: 1.2860
epoch: 15.0000

💾 Model saved to: ./emotion-recognition-model/final

🎉 All done! Your emotion recognition model is ready!


In [3]:
"""
Check if your model was saved correctly
"""
import os
from pathlib import Path

print("Checking model directory...")
print("="*60)

# Check main model directory
model_dir = Path("/kaggle/working/emotion-recognition-model")

if model_dir.exists():
    print(f"✅ Directory exists: {model_dir}")
    print(f"\nContents of {model_dir}:")
    print("-"*60)
    
    for item in sorted(model_dir.iterdir()):
        if item.is_dir():
            print(f"  📁 {item.name}/")
            # Check subdirectories
            for subitem in sorted(item.iterdir())[:5]:  # Show first 5 items
                print(f"     - {subitem.name}")
            if len(list(item.iterdir())) > 5:
                print(f"     ... and {len(list(item.iterdir())) - 5} more files")
        else:
            size = item.stat().st_size / (1024 * 1024)  # Size in MB
            print(f"  📄 {item.name} ({size:.2f} MB)")
    
    # Check for required files
    print("\n" + "="*60)
    print("Checking for required model files:")
    print("-"*60)
    
    required_files = [
        "config.json",
        "preprocessor_config.json",
        "model.safetensors",  # or pytorch_model.bin
    ]
    
    for file in required_files:
        file_path = model_dir / file
        if file_path.exists():
            print(f"  ✅ {file}")
        else:
            # Check alternative names
            if file == "model.safetensors":
                alt_path = model_dir / "pytorch_model.bin"
                if alt_path.exists():
                    print(f"  ✅ pytorch_model.bin (alternative)")
                else:
                    print(f"  ❌ {file} (MISSING!)")
            else:
                print(f"  ❌ {file} (MISSING!)")
    
    # Check if there's a 'final' subdirectory
    final_dir = model_dir / "final"
    if final_dir.exists():
        print(f"\n✅ Found 'final' subdirectory!")
        print(f"\nContents of {final_dir}:")
        print("-"*60)
        for item in sorted(final_dir.iterdir()):
            if item.is_file():
                size = item.stat().st_size / (1024 * 1024)
                print(f"  📄 {item.name} ({size:.2f} MB)")
    else:
        print(f"\n⚠️  No 'final' subdirectory found")
    
else:
    print(f"❌ Directory does NOT exist: {model_dir}")
    print("\n⚠️  This means the model was NOT saved!")
    print("\nPossible reasons:")
    print("  1. Training did not complete successfully")
    print("  2. Training is still running")
    print("  3. Model was saved to a different location")
    
    # Check working directory
    print(f"\nChecking /kaggle/working/ directory:")
    print("-"*60)
    working_dir = Path("/kaggle/working")
    if working_dir.exists():
        for item in sorted(working_dir.iterdir()):
            if item.is_dir():
                print(f"  📁 {item.name}/")
            else:
                size = item.stat().st_size / (1024 * 1024)
                print(f"  📄 {item.name} ({size:.2f} MB)")

print("\n" + "="*60)
print("CONCLUSION:")
print("="*60)

model_path = Path("/kaggle/working/emotion-recognition-model")
final_path = model_path / "final"

if final_path.exists() and (final_path / "config.json").exists():
    print("✅ Model appears to be saved correctly!")
    print(f"   Use this path: {final_path}")
elif model_path.exists() and (model_path / "config.json").exists():
    print("✅ Model is saved in main directory!")
    print(f"   Use this path: {model_path}")
else:
    print("❌ Model was NOT saved properly!")
    print("   You need to run the training script again.")

Checking model directory...
✅ Directory exists: /kaggle/working/emotion-recognition-model

Contents of /kaggle/working/emotion-recognition-model:
------------------------------------------------------------
  📁 checkpoint-2847/
     - config.json
     - model.safetensors
     - optimizer.pt
     - rng_state.pth
     - scaler.pt
     ... and 3 more files
  📁 checkpoint-3285/
     - config.json
     - model.safetensors
     - optimizer.pt
     - rng_state.pth
     - scaler.pt
     ... and 3 more files
  📁 final/
     - config.json
     - model.safetensors
     - preprocessor_config.json
     - training_args.bin
  📁 runs/
     - Jan29_17-16-50_d701060de0c2

Checking for required model files:
------------------------------------------------------------
  ❌ config.json (MISSING!)
  ❌ preprocessor_config.json (MISSING!)
  ❌ model.safetensors (MISSING!)

✅ Found 'final' subdirectory!

Contents of /kaggle/working/emotion-recognition-model/final:
------------------------------------------------